In [1]:
import pandas as pd
import math
import json
import pickle
import time

# spatial libs
import googlemaps
from geopandas import GeoDataFrame
from shapely.geometry import Point
from shapely.ops import nearest_points

# scraping libs
from bs4 import BeautifulSoup
import urllib

In [391]:
# all US cities exceeding 50K population in 2015 census
cities = pd.read_csv('us_cities_2015.csv')
# state name - abbreviation mapping
abbrev = pd.read_csv('state_abbrev.csv')

### weather stations

In [81]:
# scrape the NOAA website for a list of weather stations for which 
# there is weather data for 2015, 2016, and 2017
# there are two different ID types for each station: USAF and WBAN
# I'll primarily use USAF, but carry along WBAN too
def get_usaf_wban(link):
    href = link.get('href')
    usaf = int(href[:6])
    wban = int(href[7:12])
    return (usaf, wban)

available_usaf_dict = {}

for year in ['2015', '2016', '2017']:
    
    # use beautifulsoup to parse the HTML
    query_string = 'https://www1.ncdc.noaa.gov/pub/data/noaa/isd-lite/{}/'.format(year)
    html = urllib.urlopen(query_string).read()
    soup = BeautifulSoup(html, 'lxml')
    
    # isolate the USAF identifier from each embedded URL
    # first five aren't consistent/useful
    all_links = soup.find_all('a')[5:]
    available_usaf_yr = map(get_usaf_wban, all_links)
    available_usaf_dict[year] = available_usaf_yr

In [82]:
# ensure that I only deal with weather stations that have data in all three years
available_usaf = list(set(available_usaf_dict['2015'])&\
                      set(available_usaf_dict['2016'])&\
                      set(available_usaf_dict['2017']))

In [174]:
# all weather stations available from ncdc/noaa, with geographic locations
stations = pd.read_csv('https://www1.ncdc.noaa.gov/pub/data/noaa/isd-history.csv')

In [95]:
# cleaning weather stations df; US only, exclude stations with missing information
stations = stations[stations.CTRY=='US']
stations.dropna(axis=0, how='any', inplace=True,\
                subset=['USAF', 'WBAN', 'STATION NAME', 'STATE', 'LAT', 'LON'])
stations.drop(['ICAO', 'ELEV(M)', 'BEGIN', 'END'], axis=1, inplace=True)

# 9's represent missing value in this dataset
stations = stations[stations.USAF != 999999]
# only stations for which there is data!
mask = stations.apply(lambda row: (row['USAF'], row['WBAN']) in available_usaf, axis=1)
stations = stations.loc[mask, :]

In [97]:
stations.head()

,USAF,WBAN,STATION NAME,CTRY,STATE,LAT,LON
14466,690150,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167
14474,690190,13910,ABILENE DYESS AFB,US,TX,32.433,-99.850
15106,699604,3145,YUMA MCAS,US,AZ,32.650,-114.617
15107,700001,26492,PORTAGE GLACIER,US,AK,60.785,-148.839
15109,700197,26558,SELAWIK,US,AK,66.600,-159.986


### cities 

In [397]:
def get_abbrev(state):
    """
    maps a full US state name to its abbreviation
    """
    try:
        return abbrev[abbrev.state == state]['abbrev'].values[0]
    except:
        return 'nah'

In [398]:
# cleaning the cities df
cities['state'] = map(lambda place: place.split(', ')[1], cities.loc[:, 'city'])
cities['state_abb'] = map(lambda state: get_abbrev(state), cities.loc[:, 'state'])
cities['city'] = map(lambda place: place.split(', ')[0], cities.loc[:, 'city'])
cities['population'] = map(lambda p: int(p.replace(',', '')), cities.loc[:, 'pop'])
cities.drop(['pop'], axis=1, inplace=True)

In [399]:
# create a googlemaps API client
with open('secrets.json') as f:    
    creds = json.load(f)
    google_client = googlemaps.Client(key=creds["googlemaps"]["key"])

In [400]:
# get a latlon from google API for each city
def get_point(row):
    # query google API with city and state string; response is geolocation
    city_string = row['city'] + ', ' + row['state_abb']
    geo_response = google_client.geocode(city_string)
    loc = geo_response[0]["geometry"]["location"]
    
    # create and return a shapely point from the response
    return Point(loc['lat'], loc['lng'])

In [101]:
cities['location'] = cities.apply(get_point, axis=1)
stations['location'] = stations.apply(lambda row: Point(row.LAT, row.LON), axis=1)

In [102]:
crs = {'init': 'epsg:4326'}
geo_cities = GeoDataFrame(cities, crs=crs, geometry='location')
geo_stations = GeoDataFrame(stations, crs=crs, geometry='location')

In [103]:
geo_stations.head()

,USAF,WBAN,STATION NAME,CTRY,STATE,LAT,LON,location
14466,690150,93121,TWENTY NINE PALMS,US,CA,34.300,-116.167,POINT (34.3 -116.167)
14474,690190,13910,ABILENE DYESS AFB,US,TX,32.433,-99.850,POINT (32.433 -99.84999999999999)
15106,699604,3145,YUMA MCAS,US,AZ,32.650,-114.617,POINT (32.65 -114.617)
15107,700001,26492,PORTAGE GLACIER,US,AK,60.785,-148.839,POINT (60.785 -148.839)
15109,700197,26558,SELAWIK,US,AK,66.600,-159.986,POINT (66.59999999999999 -159.986)


In [404]:
geo_cities.head()

,city,state,state_abb,population,location
0,New York,New York,NY,8550405,POINT (40.7127837 -74.0059413)
1,Los Angeles,California,CA,3971883,POINT (34.0522342 -118.2436849)
2,Chicago,Illinois,IL,2720546,POINT (41.8781136 -87.6297982)
3,Houston,Texas,TX,2296224,POINT (29.7604267 -95.3698028)
4,Philadelphia,Pennsylvania,PA,1567442,POINT (39.9525839 -75.1652215)


### joining weather stations to cities

In [104]:
# locate the closest weather station (with full weather data) to each city
all_stations = geo_stations.location.unary_union
for i, row in geo_cities.iterrows():
    nearest_station = nearest_points(row['location'], all_stations)[1]
    station_ids = geo_stations[geo_stations.location == nearest_station][['USAF', 'WBAN']]
    # append USAF and WBAN identifier to cities data
    geo_cities.loc[i, 'USAF'] = station_ids.iloc[0, 0]
    geo_cities.loc[i, 'WBAN'] = station_ids.iloc[0, 1]

In [165]:
geo_cities['USAF'] = geo_cities.loc[:, 'USAF'].astype(int)
geo_cities['WBAN'] = geo_cities.loc[:, 'WBAN'].astype(int)

### Write and Read geostation , geocities

In [106]:
with open('cities.pickle', 'wb') as handle:
    pickle.dump(geo_cities, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('stations.pickle', 'wb') as handle:
    pickle.dump(geo_stations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [129]:
with open('cities.pickle', 'rb') as handle:
    geo_cities = pickle.load(handle)

with open('stations.pickle', 'rb') as handle:
    geo_stations = pickle.load(handle)

In [164]:
geo_cities.head()

,city,state,state_abb,population,location,USAF,WBAN
0,New York,New York,NY,8550405,POINT (40.7127837 -74.0059413),997271,99999
1,Los Angeles,California,CA,3971883,POINT (34.0522342 -118.2436849),722874,93134
2,Chicago,Illinois,IL,2720546,POINT (41.8781136 -87.6297982),998499,99999
3,Houston,Texas,TX,2296224,POINT (29.7604267 -95.3698028),720594,188
4,Philadelphia,Pennsylvania,PA,1567442,POINT (39.9525839 -75.1652215),724080,13739


In [168]:
weather_dict = {}

### pulling down weather time series

In [171]:
# 
for i, row in geo_cities.iterrows():
    place_string = '{}, {}'.format(row['city'], row['state_abb'])
    print i, ': ', place_string
    usaf = str(row['USAF'])
    wban = str(row['WBAN'])
    # some WBAN ID's are less than 5 chars; append with leading zeroes
    wban_gap = 5 - len(wban)
    wban = (wban_gap * '0') + wban
    
    all_weather = pd.DataFrame(columns=['year', 'month', 'day', 'hour', 'temp'])
    for year in ['2015', '2016', '2017']:
        # build a URL for each station and year
        query_string = 'https://www1.ncdc.noaa.gov/pub/data/noaa/isd-lite/{}/{}-{}-{}.gz'\
            .format(year, usaf, wban, year)
        # hit the NOAA API for weather data
        weather_series = pd.read_csv(query_string, compression='gzip',\
                                     delim_whitespace=True, header=None,\
                                    names=['year', 'month', 'day', 'hour', 'temp',\
                                        'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7'])
        weather_series.drop(['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7'], axis=1, inplace=True)
        # add year's worth of data to all_weather, and return
        all_weather = all_weather.append(weather_series, ignore_index=True)
        weather_dict[place_string] = all_weather

0 :  New York, NY
1 :  Los Angeles, CA
2 :  Chicago, IL
3 :  Houston, TX
4 :  Philadelphia, PA
5 :  Phoenix, AZ
6 :  San Antonio, TX
7 :  San Diego, CA
8 :  Dallas, TX
9 :  San Jose, CA
10 :  Austin, TX
11 :  Jacksonville, FL
12 :  San Francisco, CA
13 :  Indianapolis, IN
14 :  Columbus, OH
15 :  Fort Worth, TX
16 :  Charlotte, NC
17 :  Seattle, WA
18 :  Denver, CO
19 :  El Paso, TX
20 :  Detroit, MI
21 :  Washington, DC
22 :  Boston, MA
23 :  Memphis, TN
24 :  Nashville, TN
25 :  Portland, OR
26 :  Oklahoma City, OK
27 :  Las Vegas, NV
28 :  Baltimore, MD
29 :  Louisville, KY
30 :  Milwaukee, WI
31 :  Albuquerque, NM
32 :  Tucson, AZ
33 :  Fresno, CA
34 :  Sacramento, CA
35 :  Kansas City, MO
36 :  Long Beach, CA
37 :  Mesa, AZ
38 :  Atlanta, GA
39 :  Colorado Springs, CO
40 :  Virginia Beach, VA
41 :  Raleigh, NC
42 :  Omaha, NE
43 :  Miami, FL
44 :  Oakland, CA
45 :  Minneapolis, MN
46 :  Tulsa, OK
47 :  Wichita, KS
48 :  New Orleans, LA
49 :  Arlington, TX
50 :  Cleveland, OH
51 : 

In [173]:
with open('weather_series.pickle', 'wb') as handle:
    pickle.dump(weather_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [117]:
weather_dict['Philadelphia'].tail()

,year,month,day,hour,temp,x1,x2,x3,x4,x5,x6,x7
20469,2017.0,5.0,3.0,1.0,217.0,17.0,10077.0,280.0,51.0,-9999.0,-9999.0,-9999.0
20470,2017.0,5.0,3.0,2.0,211.0,22.0,10088.0,280.0,46.0,-9999.0,-9999.0,-9999.0
20471,2017.0,5.0,3.0,3.0,206.0,6.0,10092.0,270.0,57.0,-9999.0,-9999.0,-9999.0
20472,2017.0,5.0,3.0,4.0,200.0,6.0,10095.0,270.0,77.0,-9999.0,-9999.0,-9999.0
20473,2017.0,5.0,3.0,5.0,194.0,11.0,10096.0,280.0,36.0,-9999.0,-9999.0,-9999.0
